<a href="https://colab.research.google.com/github/moodlep/basic_dqn/blob/master/mlc_dqn_atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Do we need this?? 

In [1]:
!pip install ale-py

     |████████████████████████████████| 1.6 MB 4.2 MB/s 


In [1]:
from ale_py import ALEInterface

ale = ALEInterface()

In [ ]:
from ale_py.roms import Breakout

ale.loadROM(Breakout)

Below taken from [link](https://colab.research.google.com/github/GiannisMitr/DQN-Atari-Breakout/blob/master/dqn_atari_breakout.ipynb)


# Install Atari ROMs 

In [ ]:
#Make necessary imports

import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars

In [5]:
# Test a basic env: 
import gym
env = gym.make('Breakout-v4')
env.observation_space.shape, env.action_space.n

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## SB3 has some nice wrappers we can use, taken from cleanrl

In [ ]:
!pip install stable-baselines3[extra]

In [28]:
# taken from https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/atari/dqn_atari.py
from gym.spaces import Discrete
from gym.wrappers import Monitor
from stable_baselines3.common.atari_wrappers import (
    ClipRewardEnv,
    EpisodicLifeEnv,
    FireResetEnv,
    MaxAndSkipEnv,
    NoopResetEnv,
    WarpFrame,
)
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack


In [26]:
# taken from https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/atari/dqn_atari.py
def make_env(gym_id, idx):
    def thunk():
        env = gym.make(gym_id)
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        # if args.capture_video:
        #     if idx == 0:
        #         env = Monitor(env, f"videos/{experiment_name}")
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = WarpFrame(env, width=84, height=84)
        env = ClipRewardEnv(env)
        # env.seed(seed)
        # env.action_space.seed(seed)
        # env.observation_space.seed(seed)
        return env

    return thunk

In [30]:
envs = VecFrameStack(
        DummyVecEnv([make_env('Breakout-v4', 0)]),
        4,
    )

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [31]:
envs.observation_space.shape, envs.action_space.n

((84, 84, 4), 18)

In [ ]:
envs.observation_space.shape

# Coding DQN with CNNs

In [32]:
import torch
import torch.nn as nn

In [73]:
class DQN_Network(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN_Network, self).__init__()

        self.state_dim = state_dim
        self.action_dim = action_dim

        # create the dqn and target network architecture
        self.network = nn.Sequential(
            nn.Conv2d(4, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2), 
            nn.ReLU(), 
            nn.Conv2d(64, 64, 3, stride=1), 
            nn.ReLU(), 
            nn.Flatten(),
            nn.Linear(64*7*7, 256),
            nn.ReLU(),
            nn.Linear(256, self.action_dim)

        )

    def forward(self, input):
        
        state_t = torch.FloatTensor(input).permute(2, 0, 1).unsqueeze(0)
        print(state_t.shape, type(state_t))

        q_values = self.network(state_t)
        return q_values


In [61]:
state = envs.observation_space.sample()
state.shape, type(state)

((84, 84, 4), numpy.ndarray)

In [74]:
dqn = DQN_Network(envs.observation_space.shape, envs.action_space.n)

In [75]:
dqn(state)

torch.Size([1, 4, 84, 84]) <class 'torch.Tensor'>


tensor([[-7.0130, -1.4074,  1.9680, -2.8426,  7.3948,  1.0990, -2.9981,  0.7886,
         -0.9455, -7.2115,  1.6933,  4.7429,  3.7425,  2.9521,  0.8590, -0.2901,
          4.2088, -2.2177]], grad_fn=<AddmmBackward0>)